This Module is responsible for:
 

*   Finding the best number of topics using Coherence Value

*   Performing the LDA


*   Saving topics





## Imports

In [ ]:
pip install pyLDAvis

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Configurations

In [ ]:
#DEFAULT_SAVING_PATH = 'D:\\Jupyter notebooks\\Twitter-Gensim-LDA-Labelling-WE-Edited2\\'
#DATASET_PATH = 'D:\\Jupyter notebooks\\Twitter-Gensim-LDA-Labelling-WE-Edited2\\dataset-USA-stream.xlsx'
DEFAULT_SAVING_PATH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/'
OUTPUT_PATH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/'
#BIGRAM_PKL = 'Output\\bigram-uk-2000.pkl'
#TRIGRAM_PKL = 'Output\\trigram-uk-2000.pkl'
TOPICS_PKL = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/topics-MR80.pkl'
#TRIGRAM_OUTPUT = 'Output\\trigrams.csv'
#BIGRAM_OUTPUT = 'Output\\bigrams.csv'
TOP_WORDS_OUTPUT = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/toptweetwords-MR80.csv'
T_K_D = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/topkd-MR80.csv'

LDA_MODEL = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/model-MR.gensim'
DIC_PATH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/dictionary-MR.gensim'
CORPUS_PATH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/corpus-MR.pkl'
LDA_TOPICS = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/lda-topics-MR80.csv'



PREPROCESSED_TWEETS1 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/movie-pang-processed-98-12-08.csv'


NUM_TOPICS = 80
NUM_WORDS = 10
'''
IN moteghayer moshakhas minamayad ke yek topic bayad dar che 
tedad doc vojud dashte bashad ta be onvane topice mohem ghalamdad shavad
'''
NUM_TOPIC_DOC = 50
NUM_IMPORTANT_TOPICS = 100
TWEET_CONTENT_LIMIT = 50


TWEET_CONTENT = 'Tweet content'
TWEET_LANG = 'lang'
TWEET_ID = 'Tweet Id'
TWEET_FOLLOWERS = 'Followers'
TWEET_FOLLOWING = 'Following'
TWEET_FAVS = 'Favs'
TWEET_RTS = 'RTs'
TWEET_LISTED = 'Listed'


In [ ]:
import pprint
import csv
import time
from gensim import corpora, models, similarities
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from itertools import chain
from collections import Counter
import spacy
import gensim, logging
from time import time
import numpy as np
import pandas as pd
import string
from gensim.models import Word2Vec
import pickle

import nltk
from nltk.collocations import *

from gensim.models import Phrases
from gensim.models.phrases import Phraser


#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from google.colab import drive
drive.mount('/MYDRIVE', force_remount=True)

## Loading Files

In [ ]:
#Getting Ready for LDA
print("Getting Ready for LDA")
print("Loading preprocessed tweets")
dfPreprocessedTweets = pd.read_csv(PREPROCESSED_TWEETS1, header=None)
print("The preprocessed tweets were loaded")
print("The shape of dataframe is", dfPreprocessedTweets.shape)

In [ ]:
dfPreprocessedTweets.head(3)

In [ ]:
dfPreprocessedTweets.shape

In [ ]:
dfPreprocessedTweets = dfPreprocessedTweets.dropna()

In [ ]:
dfPreprocessedTweets.shape

In [ ]:
PREPROCESSED_TWEETS1

## Making list before LDA

In [ ]:
tweet_content = []
print("Converting dataframe to array")
for row in dfPreprocessedTweets.itertuples(index=False):
    tweet_content.append(row[1])
print("Converting is done successfully")

Converting dataframe to array
Converting is done successfully


In [ ]:
tweet_content = [d.split() for d in tweet_content]

## LDA

In [ ]:
DIC_PATH

'/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/dictionary-MR.gensim'

In [ ]:
CORPUS_PATH

'/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/corpus-MR.pkl'

In [ ]:
print("Creating dictionary...")
dictionary = corpora.Dictionary(tweet_content)
print("Saving dictionary...")
dictionary.save( DIC_PATH )
print("Creating corpus...")
corpus = [dictionary.doc2bow(text) for text in tweet_content]
print("Saving corpus")
pickle.dump(corpus, open( CORPUS_PATH , 'wb'))
print("Corpus saved")

Creating dictionary...
Saving dictionary...


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Creating corpus...
Saving corpus
Corpus saved


## Get the best topic number

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit=120, start=5, step=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    perplexity_values = []
    for num_topics in range(start, limit, step):
        #model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model = models.ldamodel.LdaModel(corpus=corpus,
                                    id2word=dictionary, 
                                    iterations=200, 
                                    num_topics=num_topics,
                                    update_every=1,
                                    chunksize=100,
                                    passes=20,
                                    alpha='auto',
                                    per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        perplexity_values.append(model.log_perplexity(corpus))
    return model_list, coherence_values, perplexity_values

In [ ]:
# Can take a long time to run.
limit=120; start=5; step=5;
model_list, coherence_values, perplexity_values  = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=tweet_content, start=start, limit=limit, step=step)

In [ ]:
# Show graph

x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Number of topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')

plt.show()

In [ ]:
# Print the coherence scores
i = 0
for m, cv, pe in zip(x, coherence_values, perplexity_values):
    #print(i)
    print(i, " Num Topics =", m, " has Coherence Value of", round(cv, 4), " and Perplexity of", round(pe,4))
    i = i + 1

In [ ]:
NUM_TOPICS = 64

In [ ]:
lda_model = models.ldamodel.LdaModel(corpus=corpus,
                                    id2word=dictionary, 
                                    iterations=200, 
                                    num_topics=NUM_TOPICS,
                                    update_every=1,
                                    chunksize=100,
                                    passes=20,
                                    alpha='auto',
                                    per_word_topics=True)

##Topics

In [ ]:
len(lda_model.get_topics())

64

In [ ]:
len(lda_model.show_topics(num_topics=64))

64

In [ ]:
from pprint import pprint

model_topics = lda_model.show_topics(formatted=False)
pprint(lda_model.print_topics(num_words=10))

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

In [ ]:
LDA_MODEL

'/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/model-MR.gensim'

In [ ]:
lda_model.save( LDA_MODEL )
t1 = time()
print("Building and saving model took: " )

Building and saving model took: 


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
topics = lda_model.print_topics(num_topics=80)

In [ ]:
#saving topics without text
topic_words = []
topics_df = pd.DataFrame()
for i in range(64):
    wp = lda_model.show_topic(i)
    topic_keywords = ", ".join([word for word, prop in wp])
    #tt = lda_model.get_topic_terms(i,10)
    #topics_df = topics_df.append(pd.Series([i,  [dictionary[pair[0]] for pair in tt]]), ignore_index=True)
    topics_df = topics_df.append(pd.Series([i,  topic_keywords]), ignore_index=True)
    

In [ ]:
topics_df.to_csv('/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/topkd-MR64.csv')

## Saving topics

In [ ]:
with open( TOPICS_PKL , 'wb') as to:
    pickle.dump( topics , to)

print("Topics saved")

Topics saved
